In [ ]:
import sqlite3
import pandas as pd
import random
import uuid
from faker import Faker
from datetime import timedelta

# Setup
db_path = "../data/business_finance.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
fake = Faker()

# Drop tables if they exist
tables = ["employees", "departments", "projects", "project_allocations", "financials", "salaries"]
for table in tables:
    cursor.execute(f"DROP TABLE IF EXISTS {table}")

# Create tables
cursor.executescript("""
CREATE TABLE departments (
    department_id TEXT PRIMARY KEY,
    department_name TEXT
);

CREATE TABLE employees (
    employee_id TEXT PRIMARY KEY,
    full_name TEXT,
    department_id TEXT,
    hire_date DATE,
    FOREIGN KEY(department_id) REFERENCES departments(department_id)
);

CREATE TABLE projects (
    project_id TEXT PRIMARY KEY,
    project_name TEXT,
    department_id TEXT,
    start_date DATE,
    end_date DATE,
    FOREIGN KEY(department_id) REFERENCES departments(department_id)
);

CREATE TABLE project_allocations (
    allocation_id TEXT PRIMARY KEY,
    project_id TEXT,
    employee_id TEXT,
    allocation_percentage INTEGER,
    FOREIGN KEY(project_id) REFERENCES projects(project_id),
    FOREIGN KEY(employee_id) REFERENCES employees(employee_id)
);

CREATE TABLE financials (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    month TEXT,
    department_id TEXT,
    revenue REAL,
    expenses REAL,
    profit REAL,
    margin REAL,
    FOREIGN KEY(department_id) REFERENCES departments(department_id)
);

CREATE TABLE salaries (
    salary_id TEXT PRIMARY KEY,
    employee_id TEXT,
    base_salary REAL,
    bonus REAL,
    FOREIGN KEY(employee_id) REFERENCES employees(employee_id)
);
""")

# Insert departments
departments = [(str(uuid.uuid4()), name) for name in ["Sales", "Marketing", "IT", "Finance", "HR"]]
cursor.executemany("INSERT INTO departments VALUES (?, ?)", departments)

# Insert employees
employees = []
for _ in range(50):
    dept = random.choice(departments)
    employees.append((
        str(uuid.uuid4()),
        fake.name(),
        dept[0],
        fake.date_between(start_date='-5y', end_date='today').isoformat()
    ))
cursor.executemany("INSERT INTO employees VALUES (?, ?, ?, ?)", employees)

# Insert projects
projects = []
for _ in range(20):
    dept = random.choice(departments)
    start = fake.date_between(start_date='-2y', end_date='-6m')
    end = start + timedelta(days=random.randint(90, 365))
    projects.append((
        str(uuid.uuid4()),
        fake.bs().title(),
        dept[0],
        start.isoformat(),
        end.isoformat()
    ))
cursor.executemany("INSERT INTO projects VALUES (?, ?, ?, ?, ?)", projects)

# Insert project allocations
allocations = []
for project in projects:
    num_employees = random.randint(2, 6)
    allocated_emps = random.sample(employees, num_employees)
    for emp in allocated_emps:
        allocations.append((
            str(uuid.uuid4()),
            project[0],
            emp[0],
            random.choice([25, 50, 75, 100])
        ))
cursor.executemany("INSERT INTO project_allocations VALUES (?, ?, ?, ?)", allocations)

# Insert financials
months = pd.date_range("2023-01-01", "2024-12-01", freq="MS").strftime("%Y-%m").tolist()
for month in months:
    for dept in departments:
        revenue = round(random.uniform(100000, 500000), 2)
        expenses = round(random.uniform(50000, revenue), 2)
        profit = revenue - expenses
        margin = round((profit / revenue) * 100, 2)
        cursor.execute("INSERT INTO financials (month, department_id, revenue, expenses, profit, margin) VALUES (?, ?, ?, ?, ?, ?)",
                       (month, dept[0], revenue, expenses, profit, margin))

# Insert salaries
salaries = []
for emp in employees:
    base = round(random.uniform(30000, 120000), 2)
    bonus = round(random.uniform(1000, 10000), 2)
    salaries.append((
        str(uuid.uuid4()),
        emp[0],
        base,
        bonus
    ))
cursor.executemany("INSERT INTO salaries VALUES (?, ?, ?, ?)", salaries)

# Save and close
conn.commit()
conn.close()
print(f"Database created at: {db_path}")


Database created at: business_finance.db
